In [1]:
from sklearn import ensemble, cross_validation, learning_curve, metrics 

import numpy as np
import pandas as pd
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
#from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.grid_search import GridSearchCV
import re
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
import xgboost as xgb
import pymorphy2
from sklearn.preprocessing import LabelEncoder

import pymystem3
from natasha import NamesExtractor
# pd.options.mode.chained_assignment = None

/home/amyltsev/anaconda2/envs/snakes3/lib/python3.4/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/amyltsev/anaconda2/envs/snakes3/lib/python3.4/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
/home/amyltsev/anaconda2/envs/snakes3/lib/python3.4/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions ar

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
names_data=pd.read_csv('train.csv')
names_test = pd.read_csv('test.csv')
names_data.head()

,Word,Label
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0


In [4]:
names_data.groupby('Label').describe()

Word
Label                 
0     count      90770
      unique     90769
      top       кунгфу
      freq           2
1     count      10638
      unique     10638
      top     Винклеру
      freq           1

In [5]:
names_data = pd.read_csv('train.csv')
X = names_data['Word']
y = names_data['Label']
del names_data

Генерация фич
===

In [6]:
X__all_words = set(X.str.lower())

In [9]:
words_both_labels = set(
    pd.concat([X, y], axis=1) \
        .groupby(by='Word').filter(lambda x: set(x.Label) == {0,1}) \
        .Word
)

In [10]:
morph = pymorphy2.MorphAnalyzer()
mystem = pymystem3.Mystem()
func = NamesExtractor()

def get_features(words):
    result = pd.DataFrame()
    
    result['word_len'] = words.apply(lambda w: len(w))
    
    result['pymorphy'] = words.apply(lambda x: morph.tag(x)[0])
    result['pymorphy_animacy'] = result['pymorphy'].apply(lambda x: x.animacy)
    result['pymorphy_POS'] = result['pymorphy'].apply(lambda x: x.POS)
    result['pymorphy_case'] = result['pymorphy'].apply(lambda x: x.case)
    result['pymorphy_number'] = result['pymorphy'].apply(lambda x: x.number)
    result['pymorphy_gender'] = result['pymorphy'].apply(lambda x: x.gender)
    result.drop('pymorphy', axis=1, inplace=True)
    
    columns_to_one_hot = ['pymorphy_animacy', 'pymorphy_POS', 'pymorphy_case','pymorphy_number', 'pymorphy_gender']
    for col in columns_to_one_hot:
        result[col] = LabelEncoder().fit_transform(list(result[col].fillna('nan')))
    
    result['first_letter_uppercase'] = words.apply(lambda w: (w[0].isupper()))
    result['all_uppercase'] = words.apply(lambda w: (w.isupper()))

    #russian_alphabet = 'абвгдежзийклмнопрстуфхцчшщъыьэюя-'
    #for char in russian_alphabet:
    #    names_data[char] = names['Word'].apply(lambda w: char in w.lower())
    
    allowed_vowel = set('уеёыаоэяию')
    allowed_consonant = set('йцкнгшщзхъфвпрлджчсмтьб')
    allowed_letters = allowed_vowel | allowed_consonant | set("-’'")
    
    result['allowed_chars_only'] = words.apply(lambda w: (len(set(w.lower()) - allowed_letters) == 0))
    result['vowels_count'] = words.apply(lambda w: len([c for c in w.lower() if c in allowed_vowel]))
    result['consonants_count'] = words.apply(lambda w: len([c for c in w.lower() if c in allowed_consonant]))
    
    def femine_gender(w):
        w = w.lower()
        male_gender = (w[-1] in allowed_consonant) and ((w + 'а') in X__all_words)
        fem_gender  = (w[-1] == 'а') and (w[:-1] in X__all_words)
        return male_gender or fem_gender
    result['has_feminine_gender'] = words.apply(femine_gender)
    
    ###########
    # PyMyStem
    ###########
    def name_from_pymystem(word):
        try:
            return 1 if 'имя' in mystem.analyze(word)[0]['analysis'][0]['gr'].split(',') else 0
        except:
            return 0

    def surn_from_pymystem(word):
        try:
            return 1 if 'фам' in mystem.analyze(word)[0]['analysis'][0]['gr'].split(',') else 0
        except:
            return 0
    result['name_from_pymystem'] = words.apply(name_from_pymystem)
    result['surn_from_pymystem'] = words.apply(surn_from_pymystem)
    
    ###########
    # Natasha
    ###########
    def function_natasha(word):
        return 1 if func(word) else 0
    result['natasha_person'] = words.apply(function_natasha)
    

    forbidden_chars = set('./0123456789«·»éó')
    result['has_special_char'] = words.apply(lambda w: (len(set(w.lower()) & forbidden_chars) > 0))
    
    result['in_both_classes'] = words.apply(lambda w: (w.lower() in words_both_labels))

    #result['Word'] = words
    #result['Label'] = words['Label']
    #names_data[names_data['has_invalid_char']]
    
    return result

#ddd=pd.concat([get_features(X_train), X_train, y_train], axis=1).sort_index()
#ddd[ddd.Label].head(20)

In [11]:
X__features = get_features(X)

In [12]:
X__features.tail(n=7)

,word_len,pymorphy_animacy,pymorphy_POS,pymorphy_case,pymorphy_number,pymorphy_gender,first_letter_uppercase,all_uppercase,allowed_chars_only,vowels_count,consonants_count,has_feminine_gender,name_from_pymystem,surn_from_pymystem,natasha_person,has_special_char,in_both_classes
101401,4,1,8,8,2,1,False,False,True,2,2,True,0,0,0,False,False
101402,5,1,8,4,2,1,False,False,True,3,2,True,0,0,0,False,False
101403,5,0,8,8,2,1,True,False,True,2,3,False,0,1,0,False,False
101404,5,1,8,0,2,0,False,False,True,2,3,False,0,0,0,False,False
101405,4,1,8,4,1,0,False,False,True,2,2,False,0,0,0,False,False
101406,7,1,8,0,2,0,False,False,True,3,4,False,0,0,0,False,False
101407,7,1,8,8,2,0,False,False,True,2,5,False,0,0,0,False,False


### ngrams

In [13]:
names_test__index_start = len(X)
names_test__index_start

101408

In [15]:
names_data=pd.read_csv('train.csv')

names_test__copy = names_test.copy()
names_test__copy.index += names_data.index.max() + 1
names_test__copy.index

Int64Index([101408, 101409, 101410, 101411, 101412, 101413, 101414, 101415,
            101416, 101417,
            ...
            290318, 290319, 290320, 290321, 290322, 290323, 290324, 290325,
            290326, 290327],
           dtype='int64', length=188920)

In [16]:
names_all = pd.concat([names_data, names_test__copy])
names_all[101404:101412]

,Label,Word
101404,0.0,ёлкой
101405,0.0,ёлок
101406,0.0,ёлочкой
101407,0.0,ёмкость
101408,NaN,Аалто
101409,NaN,ААР
101410,NaN,Аара
101411,NaN,Ааре


In [17]:
bigram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2,10), 
                                    max_features=5000, encoding='utf8')
for_tf_idf = bigram_vectorizer.fit_transform(names_all.Word.apply(str.lower).tolist())
tfidf_sparse = TfidfTransformer(use_idf=True).fit_transform(for_tf_idf)

In [18]:
tfidf = pd.DataFrame(tfidf_sparse.toarray(), index=names_all.index,
                     columns=['tf_idf_' + str(i) for i in range(tfidf_sparse.shape[1])])
tfidf_train = tfidf.sort_index()[:names_data.index.max()+1]
tfidf_test = tfidf.sort_index()[names_data.index.max()+1:]

names_data.shape, names_test.shape, tfidf.shape, tfidf_train.shape, tfidf_test.shape

((101408, 2), (188920, 1), (290328, 5000), (101408, 5000), (188920, 5000))

In [19]:
tfidf.tail()

,tf_idf_0,tf_idf_1,tf_idf_2,tf_idf_3,tf_idf_4,tf_idf_5,tf_idf_6,tf_idf_7,tf_idf_8,tf_idf_9,...,tf_idf_4990,tf_idf_4991,tf_idf_4992,tf_idf_4993,tf_idf_4994,tf_idf_4995,tf_idf_4996,tf_idf_4997,tf_idf_4998,tf_idf_4999
290323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.388803,0.0,0.0,0.0
290324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.388803,0.0,0.0,0.0
290325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.361601,0.0,0.0,0.0
290326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.307834,0.0,0.0,0.0
290327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.229257,0.230887,0.0,0.0,0.0,0.0,0.328147,0.0,0.0,0.0


In [20]:
del names_data, names_all

In [21]:
X__features_tfidf = pd.concat([X__features, tfidf_train], axis=1)

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(X__features_tfidf, y, test_size=.2, random_state=42)

In [23]:
X_train.head()

,word_len,pymorphy_animacy,pymorphy_POS,pymorphy_case,pymorphy_number,pymorphy_gender,first_letter_uppercase,all_uppercase,allowed_chars_only,vowels_count,...,tf_idf_4990,tf_idf_4991,tf_idf_4992,tf_idf_4993,tf_idf_4994,tf_idf_4995,tf_idf_4996,tf_idf_4997,tf_idf_4998,tf_idf_4999
63786,10,0,8,0,1,0,False,False,True,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48947,17,0,8,6,2,1,False,False,True,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83846,11,1,8,6,1,1,False,False,True,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59674,9,1,8,2,2,1,False,False,True,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13575,9,1,8,4,1,1,True,False,True,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
y_train.head()

63786    0
48947    0
83846    0
59674    0
13575    0
Name: Label, dtype: int64

In [25]:
X_train.shape, y_train.shape

((81126, 5017), (81126,))

Обучение модели
===

In [26]:
d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)

In [27]:
print('Training ...')

params = {'max_depth': 10, 
          'min_child_weight': 50, 
          #'subsample': 0.9,
          'colsample_bytree': 0.9,  
          'colsample_bylevel': 0.6,
          'lambda': 0.1,
          'alpha': 0.1,
          'eta': 0.5, 
          'objective': 'binary:logistic',
          'eval_metric': 'auc',
          'silent': 1,
          'seed': 17
         }

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
model = xgb.train(params=params, dtrain=d_train, num_boost_round=40000, 
                  evals=watchlist, early_stopping_rounds=500, verbose_eval=10)

Training ...
[0]	train-auc:0.921544	valid-auc:0.912341
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 500 rounds.
[10]	train-auc:0.943325	valid-auc:0.935371
[20]	train-auc:0.947072	valid-auc:0.936432
[30]	train-auc:0.949122	valid-auc:0.936616
[40]	train-auc:0.951316	valid-auc:0.936691
[50]	train-auc:0.953046	valid-auc:0.937209
[60]	train-auc:0.954478	valid-auc:0.937624
[70]	train-auc:0.955301	valid-auc:0.937826
[80]	train-auc:0.956212	valid-auc:0.937731
[90]	train-auc:0.957692	valid-auc:0.938416
[100]	train-auc:0.958724	valid-auc:0.938474
[110]	train-auc:0.959756	valid-auc:0.938911
[120]	train-auc:0.96039	valid-auc:0.938972
[130]	train-auc:0.961081	valid-auc:0.938979
[140]	train-auc:0.961799	valid-auc:0.939317
[150]	train-auc:0.962417	valid-auc:0.939435
[160]	train-auc:0.963091	valid-auc:0.939603
[170]	train-auc:0.963582	valid-auc:0.939452
[180]	train-auc:0.964303	valid-auc:0.939563
[190]	train-auc:0.9

In [35]:
d_train_full = xgb.DMatrix(X__features_tfidf, label=y)
print('AUC XgBoost: ', roc_auc_score(y, model.predict(d_train_full)))

AUC XgBoost:  0.978173460301


In [33]:
model.save_model('xgb__pymorphy_natasha.model')

Классификация теста
===

In [36]:
X_test = names_test['Word']
print(X_test.shape, names_test.shape)
X_test__features = get_features(X_test)
X_test__features.head()

(188920,) (188920, 1)


,word_len,pymorphy_animacy,pymorphy_POS,pymorphy_case,pymorphy_number,pymorphy_gender,first_letter_uppercase,all_uppercase,allowed_chars_only,vowels_count,consonants_count,has_feminine_gender,name_from_pymystem,surn_from_pymystem,natasha_person,has_special_char,in_both_classes
0,5,2,1,7,2,3,True,False,True,3,2,False,0,0,0,False,False
1,3,2,17,7,0,2,True,True,True,2,1,False,0,0,0,False,False
2,4,1,8,4,2,1,True,False,True,3,1,True,0,0,0,False,False
3,4,1,8,8,2,0,True,False,True,3,1,False,0,0,0,False,False
4,5,0,8,8,2,1,True,False,True,3,2,True,0,0,1,False,False


In [37]:
X_test__features.shape

(188920, 17)

In [38]:
tfidf_test__copy = tfidf_test.copy()
tfidf_test__copy.index -= tfidf_test__copy.index.min()
tfidf_test__copy.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            188910, 188911, 188912, 188913, 188914, 188915, 188916, 188917,
            188918, 188919],
           dtype='int64', length=188920)

In [39]:
X_test__features__tfidf = pd.concat([X_test__features, tfidf_test__copy], axis=1)
X_test__features__tfidf.shape

(188920, 5017)

In [40]:
d_test = xgb.DMatrix(X_test__features__tfidf)
predictions = model.predict(d_test)
predictions

array([ 0.11589072,  0.16156481,  0.01235403, ...,  0.00327116,
        0.00063115,  0.00093985], dtype=float32)

In [48]:
submission = pd.DataFrame({'Id': range(0, len(predictions)), 'Prediction': predictions})
submission.to_csv('myltsev_submission.csv', index=False)